# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu 1500km et simu 1000km.

## Sims

In [ ]:
largeLAM_dir='../../../JZ_simu_outputs/LAM/LAM_1500km'
smallLAM_dir='../../../JZ_simu_outputs/LAM/noirr_2010_2022'

In [ ]:
lon_min=-18
lon_max=11
lat_min=29
lat_max=52
date_min = '2010-01-01'
date_max = '2012-12-31'

In [ ]:
#sim
# filename = '{}/ATM/TS*.nc'.format(largeLAM_dir)
filename = '{}/ATM/MO/*.nc'.format(largeLAM_dir)

sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = '1500km'
sim = sim0.rename({'time_counter':'time'})
sim = sim.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
sim = sim.sel(time=slice(date_min, date_max))

sim['evap'] = sim['evap'] *3600 * 24
sim['evap'].attrs['units'] = 'mm/d'

sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm/d'

sim['fluxsens']= -sim['sens']

sim['netrad'] = sim['LWdnSFC'] - sim['LWupSFC'] + sim['SWdnSFC'] - sim['SWupSFC']
sim['netrad'].attrs['units'] = 'W/m2'

sim['P - E'] = sim['precip'] - sim['evap']
sim['P - E'].attrs['units'] = 'mm/d'

sim

In [ ]:
#smaller LAM (original)
filename = '{}/ATM/TS*.nc'.format(smallLAM_dir)
# filename = '{}/ATM/MO/*.nc'.format(smallLAM_dir)

sim0small = xr.open_mfdataset(filename)
sim0small.attrs['name'] = '1000km'
small = sim0small.rename({'time_counter':'time'})
small = small.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
small = small.sel(time=slice(date_min, date_max))

small['evap'] = small['evap'] *3600 * 24
small['evap'].attrs['units'] = 'mm/d'

small['precip'] = small['precip'] *3600 * 24
small['precip'].attrs['units'] = 'mm/d'

small['fluxsens']= -small['sens']

small['netrad'] = small['LWdnSFC'] - small['LWupSFC'] + small['SWdnSFC'] - small['SWupSFC']
small['netrad'].attrs['units'] = 'W/m2'

small['P - E'] = small['precip'] - small['evap']
small['P - E'].attrs['units'] = 'mm/d'

small


## Obs

In [ ]:
#open ERA5 file
filename = '../../../obs/ERA5_eval_2010_2022.nc'
era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip'
        }
era = era.rename(dict)
era=era.sel(time=slice(date_min, date_max))

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'
era['cldt'] = era['tcc']*100
era['cldt'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era


In [ ]:
#gleam
filename='../../../obs/evap/E_1980-2022_GLEAM_v3.8a_MO.nc'
gleam0=xr.open_dataset(filename)
gleam0.attrs['name']='GLEAM'
gleam = gleam0.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#adapt period
gleam=gleam.sel(time=slice(date_min, date_max))
#name gleam vars like in LMDZOR outputs
dict =  {
        'E':'evap'
        }
gleam = gleam.rename(dict)
#convert mm/month to mm/day
gleam['evap']=gleam['evap']/30
gleam['evap'].attrs['units']='mm/d'
gleam

In [ ]:
#FluxCom
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom0=xr.open_mfdataset(filename)
fluxcom=fluxcom0.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))
fluxcom=fluxcom.sel(time=slice(date_min, date_max))

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice(date_min, date_max))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_min,lat_max))
gpcc=gpcc1.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max,lat_min))
# gpcc2=gpcc.sel(lon=slice(0,lon_max+0.25),lat=slice(lat_max,lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < lon_max+0.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25))

gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

In [ ]:
#open tqruv file
filename='../../../obs/TQRUV_850_2010_2022_monthly.nc'
tqruv = xr.open_dataset(filename)
tqruv.attrs['name'] = 'ERA5'
tqruv = tqruv.sel(longitude=slice(lon_min,lon_max),latitude=slice(lat_max,lat_min))
tqruv = tqruv.sel(time=slice(date_min, date_max))
#rename longitude and latitude to lon and lat
tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

#rename all variables to add 850 in name
tqruv['u850'] = tqruv['u']
tqruv['v850'] = tqruv['v']
tqruv['q850'] = tqruv['q']
tqruv['t850'] = tqruv['t']
tqruv['r850'] = tqruv['r']
tqruv

# Masking, interpolation

In [ ]:
# Select months
# months=[7]

# orc = orc.sel(time=orc['time.month'].isin(months))
# orcirr = orcirr.sel(time=orcirr['time.month'].isin(months))

# sim = sim.sel(time=sim['time.month'].isin(months))
# small = small.sel(time=small['time.month'].isin(months))

# era = era.sel(time=era['time.month'].isin(months))
# fluxcom=fluxcom.sel(time=fluxcom['time.month'].isin(months))

In [ ]:
#continental fraction mask (LMDZ outputs)
con_mask=sim['contfracATM']>0.9

#create a masks of data points where irrig_frac >5%
# irr_mask = (orcirr['irrigmap_dyn']/orcirr['Areas'])>0.05
irr_mask = con_mask

ip_mask=polygon_to_mask(sim, iberic_peninsula)
# ip_mask_orc=polygon_to_mask(orc, iberic_peninsula)

In [ ]:
small_sim=small.interp_like(sim)

In [ ]:
cont_sim=sim.where(con_mask)
cont_small=small_sim.where(con_mask)

ip_sim = sim.where(ip_mask['mask'], drop=False).where(con_mask)
ip_small = small_sim.where(ip_mask['mask'], drop=False).where(con_mask)

# ip_orc=orc.where(ip_mask_orc['mask'], drop=False)

In [ ]:
sim_era = sim.interp_like(era)
sim_gpcc = sim.interp_like(gpcc)
sim_gleam = sim.interp_like(gleam)
sim_fluxcom = sim.interp_like(fluxcom)
sim_tqruv = sim.interp_like(tqruv)

small_era = small.interp_like(era)
small_gpcc = small.interp_like(gpcc)
small_gleam = small.interp_like(gleam)
small_fluxcom = small.interp_like(fluxcom)
small_tqruv = small.interp_like(tqruv)

ip_sim_era = ip_sim.interp_like(era)
ip_sim_gpcc = ip_sim.interp_like(gpcc)
ip_sim_gleam = ip_sim.interp_like(gleam)
ip_sim_fluxcom = ip_sim.interp_like(fluxcom)
ip_sim_tqruv = ip_sim.interp_like(tqruv)

ip_small_era = ip_small.interp_like(era)
ip_small_gpcc = ip_small.interp_like(gpcc)
ip_small_gleam = ip_small.interp_like(gleam)
ip_small_fluxcom = ip_small.interp_like(fluxcom)
ip_small_tqruv = ip_small.interp_like(tqruv)

In [ ]:
# irrorc=orc.where(irr_mask)
# irrorcirr=orcirr.where(irr_mask)

In [ ]:
# era
int_era=era.interp_like(sim)
cont_era=int_era.where(con_mask)
ip_era = cont_era.where(ip_mask['mask'], drop=False)
# orc_era=era.interp_like(orc)

In [ ]:
#gleam
int_gleam=gleam.interp_like(sim)
cont_gleam=gleam.interp_like(sim).where(con_mask)
ip_gleam = cont_gleam.where(ip_mask['mask'], drop=False)
# irr_era=era.interp_like(orc).where(irr_mask)
# irr_gleam=gleam.interp_like(orc).where(irr_mask)

In [ ]:
#fluxcom
int_fluxcom=fluxcom.interp_like(sim)
cont_fluxcom=fluxcom.interp_like(sim).where(con_mask)
ip_fluxcom = cont_fluxcom.where(ip_mask['mask'], drop=False)
# orc_fluxcom=fluxcom.interp_like(orc)

In [ ]:
#gpcc
int_gpcc=gpcc.interp_like(sim)
cont_gpcc=gpcc.interp_like(sim).where(con_mask)
ip_gpcc = cont_gpcc.where(ip_mask['mask'], drop=False)

In [ ]:
int_tqruv = tqruv.interp_like(sim)
cont_tqruv = tqruv.interp_like(sim).where(con_mask)
ip_tqruv = cont_tqruv.where(ip_mask['mask'], drop=False)

# Maps

In [ ]:
#Display map of var for ds
var='precip'
ds=gpcc
#select only year 2014
color_map=reds
max_value=8

map_ave(ds, var, cmap=color_map, vmax=max_value, hex=False)

In [ ]:
#Diff map
var='evap'
max_value=2
ds1=ip_sim
ds2=ip_small

map_diff_ave(ds1, ds2, var, vmax=max_value, hex=False, sig=False)

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
map_seasons(diff, cmap=emb, vmax=max_value, title=title, hex=False)

In [ ]:
#map for 4 seasons
var='precip'
ds1=cont_sim
ds2=cont_era
max_value=2
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'P - E'
min_value=None
max_value=None
color_map=emb_neutral
ds1=sim
ds2=small
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map)

In [ ]:
#Display map of var for ds on restricted area
var='evap'
ds = sim
latmin=36.7
latmax=36.9
lonmin=-6.4
lonmax=-6.2

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb_neutral, vmax=None, title=var)

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

# Time series

In [ ]:
var='fluxsens'

ds1=ip_sim
ds2=ip_small
ds3=ip_era

ds3=ip_fluxcom

ds4=ip_gpcc

# ds4=ip_fluxcom
# ds5=ip_gleam

# ds1=ip_orc
# ds2=ip_orcirr
# ds3=ip_fluxcom_orc

ds_list=[ds1, ds2, ds3]#, ds5]
# ds_list=[ds4]

year_max=2012
time_series_ave(ds_list, var, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_max=year_max, title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))